                               Название банка            Unnamed: 1  \
0    Оборотная ведомость по балансовым счетам                   NaN   
1        за период с 01.01.2016 по 31.12.2016                   NaN   
2                                    по банку                   NaN   
3                                         NaN                   NaN   
4                         2017-01-01 00:00:00                   NaN   
..                                        ...                   ...   
620                                        95                     0   
621                                      9600                     0   
622                                        96                     0   
623                                 ПО КЛАССУ                     0   
624                                    БАЛАНС  6870787831268.599609   

               Unnamed: 2            Unnamed: 3                    \
0                     NaN                   NaN               NaN   
1        

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_19941/864237210.py:2: SyntaxWarning: invalid escape sequence '\d'
  r = re.compile('КЛАСС  \d|ПО КЛАССУ')


,unid,active,passive,debit,credit
8,1019,74975624165.882996,0,15640681705689,14312543941884
9,1011,58496794456.711998,0,689834244241.75,34144530376697
10,1020,16521001749.354,0,654759955289.73999,3865541431398.299805
11,1030,20431402379.548,0,9951931131495.599609,5748850039946
12,1040,115251239713.380005,0,9668096244082,9866340607336.699219
...,...,...,...,...,...
67,1816,0,0,138846482.71942,177337504.24508
68,1819,0,2787326.296643,105450079465.990005,132478906813.240005
69,1921,0,143307715.03092,1164296070.6193,1132735512.9409
70,1923,0,7766063985.6158,16420383959.135,8205230237.7022
